# Updated

In [1]:
import os.path
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split

# -------------------------
# Config
# -------------------------
LABEL_COL = 'Cancer_lbl'
ID_COL    = 'pid'

FEATURES_STLMD  = ['sct_long_dia','part_solid','ground_glass','solid','Upper_Lobe','Spiculation','age','sex']
FEATURES_STLM   = ['sct_long_dia','part_solid','ground_glass','solid','Upper_Lobe','Spiculation']
FEATURES_STPGLM = ['sct_long_dia','part_solid','ground_glass','Upper_Lobe','Spiculation']

# ≥ cutpoints for binning (age & size)
CUTS_CONT = {
    "age": [56, 58, 63, 68, 73],
    "sct_long_dia": [4, 6, 8, 10, 12, 15, 20, 30],
}

# binary passthroughs (copied as-is)
BIN_PASSTHROUGH_STLMD  = ['part_solid','ground_glass','solid','Upper_Lobe','Spiculation','sex']
BIN_PASSTHROUGH_STLM   = ['part_solid','ground_glass','solid','Upper_Lobe','Spiculation']
BIN_PASSTHROUGH_STPGLM = ['part_solid','ground_glass','Upper_Lobe','Spiculation']

# Data files
CSV1 = '/data/usr/ft42/CVIT_XAI/LungRADS_Modeling/NLST_Statistics/ml_dataset/nlst_ct_nodule_df_set1.csv'
CSV2 = '/data/usr/ft42/CVIT_XAI/LungRADS_Modeling/NLST_Statistics/ml_dataset/nlst_ct_nodule_df_set2.csv'

# -------------------------
# Helpers
# -------------------------
def _norm_text(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    s = re.sub(r'\s+', ' ', s)
    s = s.replace('_','-').replace('–','-').replace('—','-')
    return s

def normalize_and_encode(df: pd.DataFrame) -> pd.DataFrame:
    """
    - Canonicalize gender to {'male','female'}
    - Canonicalize Nodule_Type to {'solid','ground-glass','part-solid'} using common aliases
    - Create binaries: sex, part_solid, ground_glass, solid
    - Coerce Upper_Lobe/Spiculation to ints if present
    - Drop rows with unmapped gender/Nodule_Type; warn on non-exclusive types
    """
    df = df.copy()

    # gender
    g = df['gender'].apply(_norm_text)
    df['gender'] = g.map({'male':'male','m':'male','female':'female','f':'female'})

    # nodule type
    t = df['Nodule_Type'].apply(_norm_text)
    mapping = {
        'solid':'solid', 'sld':'solid',

        'ground-glass':'ground-glass', 'ground glass':'ground-glass',
        'ggo':'ground-glass', 'non-solid':'ground-glass',
        'non solid':'ground-glass', 'nonsolid':'ground-glass',

        'part-solid':'part-solid', 'part solid':'part-solid',
        'semi-solid':'part-solid', 'semisolid':'part-solid', 'subsolid':'part-solid',
    }
    df['Nodule_Type'] = t.map(mapping)

    # drop unmapped
    bad = df['gender'].isna() | df['Nodule_Type'].isna()
    if bad.any():
        print(f"[filter] Dropping {int(bad.sum())} rows with unmapped gender or Nodule_Type")
        # Uncomment to inspect:
        # print(df.loc[bad, [ID_COL,'gender','Nodule_Type']].head(10))
        df = df.loc[~bad].copy()

    # binaries
    df['sex']          = df['gender'].map({'male':0, 'female':1}).astype(int)
    df['part_solid']   = (df['Nodule_Type'] == 'part-solid').astype(int)
    df['ground_glass'] = (df['Nodule_Type'] == 'ground-glass').astype(int)
    df['solid']        = (df['Nodule_Type'] == 'solid').astype(int)

    # passthrough ints
    for b in ['Upper_Lobe','Spiculation']:
        if b in df.columns:
            df[b] = pd.to_numeric(df[b], errors='ignore')
            df[b] = pd.to_numeric(df[b], errors='coerce').fillna(0).astype(int)

    # sanity: exactly one nodule type flag
    onehot_sum = df[['solid','ground_glass','part_solid']].sum(axis=1)
    if (onehot_sum != 1).any():
        print(f"[warn] {int((onehot_sum != 1).sum())} rows have non-exclusive nodule types; please inspect.")

    return df

def to_fastrisk_y(y_raw, pos_label=1) -> np.ndarray:
    """Return 1-D np.ndarray[float] with labels in {-1.0, +1.0}."""
    y_arr = np.asarray(y_raw).ravel()
    uniq = set(np.unique(y_arr))
    if uniq <= {0, 1}:
        return (2 * y_arr - 1).astype(float)
    return np.where(y_arr == pos_label, 1.0, -1.0).astype(float)

def make_ge_bins(df: pd.DataFrame, feature: str, cuts: list[float]) -> pd.DataFrame:
    """
    Build columns like 'age >= 73' or 'sct_long_dia >= 20' (ASCII >=)
    """
    out = pd.DataFrame(index=df.index)
    vals = pd.to_numeric(df[feature], errors='coerce')
    for c in cuts:
        col = f"{feature} >= {c:g}"
        out[col] = (vals >= float(c)).astype(int)
    return out

def build_binary_matrix(X_df: pd.DataFrame,
                        feature_cuts: dict[str, list[float]],
                        passthrough_binary: list[str]) -> pd.DataFrame:
    mats = []
    for feat, cuts in feature_cuts.items():
        if feat not in X_df.columns:
            raise KeyError(f"Missing feature for binning: {feat}")
        mats.append(make_ge_bins(X_df, feat, cuts))
    for feat in passthrough_binary:
        if feat not in X_df.columns:
            raise KeyError(f"Missing binary feature: {feat}")
        col = pd.Series(pd.to_numeric(X_df[feat], errors='coerce')).fillna(0).astype(int)
        mats.append(pd.DataFrame({feat: col}, index=X_df.index))
    return pd.concat(mats, axis=1)

def binarize_and_align_custom(X_train_df: pd.DataFrame,
                              X_val_df: pd.DataFrame,
                              X_test_df: pd.DataFrame,
                              feature_cuts: dict[str, list[float]],
                              passthrough_binary: list[str]) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Binarize each split with ≥ cuts, then align val/test to training columns.
    """
    X_train_bin = build_binary_matrix(X_train_df, feature_cuts, passthrough_binary)
    X_val_bin   = build_binary_matrix(X_val_df,   feature_cuts, passthrough_binary)
    X_test_bin  = build_binary_matrix(X_test_df,  feature_cuts, passthrough_binary)

    def _align_like_train(train_bin_df: pd.DataFrame, other_bin_df: pd.DataFrame) -> pd.DataFrame:
        cols = list(train_bin_df.columns)
        return other_bin_df.reindex(columns=cols, fill_value=0)

    X_val_bin  = _align_like_train(X_train_bin, X_val_bin)
    X_test_bin = _align_like_train(X_train_bin, X_test_bin)

    # sanity
    assert list(X_val_bin.columns)  == list(X_train_bin.columns)
    assert list(X_test_bin.columns) == list(X_train_bin.columns)
    return X_train_bin, X_val_bin, X_test_bin

def prepare_data(df: pd.DataFrame, feature_cols: list[str], label_col: str):
    X = df[feature_cols]
    y = df[label_col]
    return X, y

# -------------------------
# Load, normalize/encode
# -------------------------
df1 = pd.read_csv(CSV1)
df2 = pd.read_csv(CSV2)

df1 = normalize_and_encode(df1)
df2 = normalize_and_encode(df2)

# -------------------------
# Patient-level stratified split (on df1)
# -------------------------
patients = df1[[ID_COL, LABEL_COL]].drop_duplicates()
train_patients, val_patients = train_test_split(
    patients,
    test_size=0.2,
    stratify=patients[LABEL_COL],
    random_state=42
)
train_df = df1[df1[ID_COL].isin(train_patients[ID_COL])]
val_df   = df1[df1[ID_COL].isin(val_patients[ID_COL])]

# ============================================================
# STLMD (includes age & sex)  —  bins: age >= c, sct_long_dia >= c
# ============================================================
X_train_STLMD_df, y_train_STLMD_raw = prepare_data(train_df, FEATURES_STLMD, LABEL_COL)
X_val_STLMD_df,   y_val_STLMD_raw   = prepare_data(val_df,   FEATURES_STLMD, LABEL_COL)
X_test_STLMD_df,  y_test_STLMD_raw  = prepare_data(df2,      FEATURES_STLMD, LABEL_COL)

X_train_STLMD_bin, X_val_STLMD_bin, X_test_STLMD_bin = binarize_and_align_custom(
    X_train_STLMD_df, X_val_STLMD_df, X_test_STLMD_df,
    feature_cuts={"age": CUTS_CONT["age"], "sct_long_dia": CUTS_CONT["sct_long_dia"]},
    passthrough_binary=BIN_PASSTHROUGH_STLMD
)

y_train_STLMD = to_fastrisk_y(y_train_STLMD_raw, pos_label=1)
y_val_STLMD   = to_fastrisk_y(y_val_STLMD_raw,   pos_label=1)
y_test_STLMD  = to_fastrisk_y(y_test_STLMD_raw,  pos_label=1)

X_train_STLMD = X_train_STLMD_bin.to_numpy(dtype=float)
X_val_STLMD   = X_val_STLMD_bin.to_numpy(dtype=float)
X_test_STLMD  = X_test_STLMD_bin.to_numpy(dtype=float)

# ============================================================
# STLM (no age/sex) — bins: sct_long_dia >= c
# ============================================================
X_train_STLM_df, y_train_STLM_raw = prepare_data(train_df, FEATURES_STLM, LABEL_COL)
X_val_STLM_df,   y_val_STLM_raw   = prepare_data(val_df,   FEATURES_STLM, LABEL_COL)
X_test_STLM_df,  y_test_STLM_raw  = prepare_data(df2,      FEATURES_STLM, LABEL_COL)

X_train_STLM_bin, X_val_STLM_bin, X_test_STLM_bin = binarize_and_align_custom(
    X_train_STLM_df, X_val_STLM_df, X_test_STLM_df,
    feature_cuts={"sct_long_dia": CUTS_CONT["sct_long_dia"]},
    passthrough_binary=BIN_PASSTHROUGH_STLM
)

y_train_STLM = to_fastrisk_y(y_train_STLM_raw, pos_label=1)
y_val_STLM   = to_fastrisk_y(y_val_STLM_raw,   pos_label=1)
y_test_STLM  = to_fastrisk_y(y_test_STLM_raw,  pos_label=1)

X_train_STLM = X_train_STLM_bin.to_numpy(dtype=float)
X_val_STLM   = X_val_STLM_bin.to_numpy(dtype=float)
X_test_STLM  = X_test_STLM_bin.to_numpy(dtype=float)

# ============================================================
# STPGLM (subset features) — bins: sct_long_dia >= c
# ============================================================
X_train_STPGLM_df, y_train_STPGLM_raw = prepare_data(train_df, FEATURES_STPGLM, LABEL_COL)
X_val_STPGLM_df,   y_val_STPGLM_raw   = prepare_data(val_df,   FEATURES_STPGLM, LABEL_COL)
X_test_STPGLM_df,  y_test_STPGLM_raw  = prepare_data(df2,      FEATURES_STPGLM, LABEL_COL)

X_train_STPGLM_bin, X_val_STPGLM_bin, X_test_STPGLM_bin = binarize_and_align_custom(
    X_train_STPGLM_df, X_val_STPGLM_df, X_test_STPGLM_df,
    feature_cuts={"sct_long_dia": CUTS_CONT["sct_long_dia"]},
    passthrough_binary=BIN_PASSTHROUGH_STPGLM
)

y_train_STPGLM = to_fastrisk_y(y_train_STPGLM_raw, pos_label=1)
y_val_STPGLM   = to_fastrisk_y(y_val_STPGLM_raw,   pos_label=1)
y_test_STPGLM  = to_fastrisk_y(y_test_STPGLM_raw,  pos_label=1)

X_train_STPGLM = X_train_STPGLM_bin.to_numpy(dtype=float)
X_val_STPGLM   = X_val_STPGLM_bin.to_numpy(dtype=float)
X_test_STPGLM  = X_test_STPGLM_bin.to_numpy(dtype=float)

# -------------------------
# Quick hygiene checks
# -------------------------
def _chk(Xtr, ytr, Xv, yv, Xte, yte, name):
    assert Xtr.shape[0] == ytr.shape[0] and Xv.shape[0] == yv.shape[0] and Xte.shape[0] == yte.shape[0], f"row mismatch in {name}"
    assert set(np.unique(ytr)) <= {-1.0, 1.0} and set(np.unique(yv)) <= {-1.0, 1.0} and set(np.unique(yte)) <= {-1.0, 1.0}, f"bad labels in {name}"
    print(f"{name:7s} -> X_train {Xtr.shape}, X_val {Xv.shape}, X_test {Xte.shape}")

_chk(X_train_STLMD, y_train_STLMD, X_val_STLMD, y_val_STLMD, X_test_STLMD, y_test_STLMD, "STLMD")
_chk(X_train_STLM,  y_train_STLM,  X_val_STLM,  y_val_STLM,  X_test_STLM,  y_test_STLM,  "STLM")
_chk(X_train_STPGLM,y_train_STPGLM,X_val_STPGLM,y_val_STPGLM,X_test_STPGLM,y_test_STPGLM,"STPGLM")

# Optional: feature names for each design
FEATURE_NAMES_STLMD  = list(X_train_STLMD_bin.columns)
FEATURE_NAMES_STLM   = list(X_train_STLM_bin.columns)
FEATURE_NAMES_STPGLM = list(X_train_STPGLM_bin.columns)


STLMD   -> X_train (13324, 19), X_val (3282, 19), X_test (16425, 19)
STLM    -> X_train (13324, 13), X_val (3282, 13), X_test (16425, 13)
STPGLM  -> X_train (13324, 12), X_val (3282, 12), X_test (16425, 12)


In [2]:
FEATURE_NAMES_STLMD

['age >= 56',
 'age >= 58',
 'age >= 63',
 'age >= 68',
 'age >= 73',
 'sct_long_dia >= 4',
 'sct_long_dia >= 6',
 'sct_long_dia >= 8',
 'sct_long_dia >= 10',
 'sct_long_dia >= 12',
 'sct_long_dia >= 15',
 'sct_long_dia >= 20',
 'sct_long_dia >= 30',
 'part_solid',
 'ground_glass',
 'solid',
 'Upper_Lobe',
 'Spiculation',
 'sex']

In [5]:
# ============================
# Training + Evaluation (with Riskomon export)
# ============================
import os, time, json, math
from typing import List, Any, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

OUTDIR = "./FasterRisk_STLMD_Outputs"
os.makedirs(OUTDIR, exist_ok=True)

# ---- knobs you can tune ----
SPARSITY_K        = 5
PARENT_SIZE       = 50

# Rashomon / diversity knobs (documented in FasterRisk)
GAP_TOLERANCE     = 0.15   # larger -> accept more near-optimal models
SELECT_TOP_M      = 100    # keep more diverse supports
MAX_ATTEMPTS      = 200    # try more swaps/diversification

# choose which designs to run
DESIGNS_TO_RUN = ["STLMD"]   # e.g. ["STLMD","STLM","STPGLM"]

# ROC subplot grid (50 per page)
SUBPLOT_ROWS = 10
SUBPLOT_COLS = 5

# ---------------------------------------------
# Version-robust optimizer creation
# ---------------------------------------------
def make_optimizer(X, y,
                   k=SPARSITY_K, parent_size=PARENT_SIZE,
                   gap_tolerance=GAP_TOLERANCE,
                   select_top_m=SELECT_TOP_M,
                   max_attempts=MAX_ATTEMPTS,
                   want_intercept=True):
    
    # (1) gap + select_top_m + maxAttempts (camelCase)
   
        return RiskScoreOptimizer(
            X=X, y=y, k=k, parent_size=parent_size,
            gap_tolerance=gap_tolerance,
            select_top_m=select_top_m,
            maxAttempts=max_attempts)

 


# ---------------------------------------------
# get_models() return format helper (2- or 3-tuple)
# ---------------------------------------------
def extract_models(ret):
    if isinstance(ret, tuple) and len(ret) == 2:
        multipliers, integer_mat = ret
        integer_mat = np.asarray(integer_mat, dtype=int)
        beta0_int = integer_mat[:, 0]
        betas_int = integer_mat[:, 1:]
        return np.asarray(multipliers, float), beta0_int, betas_int
    elif isinstance(ret, tuple) and len(ret) == 3:
        multipliers, beta0_int, betas_int = ret
        return (np.asarray(multipliers, float),
                np.asarray(beta0_int, int),
                np.asarray(betas_int, int))
    else:
        raise RuntimeError("Unexpected return format from get_models()")

# ---------------------------------------------
# Metrics + ROC plotting
# ---------------------------------------------
def model_probs(mult: float, b0: float, betas: np.ndarray, X: np.ndarray) -> np.ndarray:
    z = b0 + X @ betas.astype(float)
    return 1.0 / (1.0 + np.exp(-mult * z))

def compute_model_metrics(multipliers, intercepts, coef_matrix, X, y):
    aucs, accs, n_terms, probs_list = [], [], [], []
    y01 = ((y + 1.0) / 2.0)  # {-1,+1} -> {0,1}
    for m, b0, w in zip(multipliers, intercepts, coef_matrix):
        w = np.asarray(w, dtype=int)
        p = model_probs(float(m), float(b0), w, X)
        yhat = (p >= 0.5).astype(int)
        acc = (yhat == y01).mean()
        fpr, tpr, _ = roc_curve(y01, p)
        auc_val = auc(fpr, tpr)
        aucs.append(float(auc_val))
        accs.append(float(acc))
        n_terms.append(int((w != 0).sum()))
        probs_list.append(p)
    return np.asarray(aucs), np.asarray(accs), np.asarray(n_terms), probs_list

def print_model_summaries(multipliers, intercepts, coef_matrix, feature_names, train_aucs, test_aucs):
    width = max(2, len(str(len(multipliers))))
    print("\n=== Per-model summaries ===")
    for i, (m, b0, w) in enumerate(zip(multipliers, intercepts, coef_matrix)):
        card = f"{i+1:0{width}d}"
        w = np.asarray(w, dtype=int)
        nz = np.flatnonzero(w)
        feats = [f"{feature_names[j]}({int(w[j]):+d})" for j in nz]
        feats_str = ", ".join(feats) if feats else "(no terms)"
        print(f"[Card {card}] b0={float(b0):+.3f}, mult={float(m):.3f}, #terms={len(nz)}")
        print(f"  Train AUC: {train_aucs[i]:.3f} | Test AUC: {test_aucs[i]:.3f}")
        print(f"  Features : {feats_str}")

def plot_roc_subplots_all(y_true01, probs_list, labels, rows, cols, title_prefix, out_prefix):
    from math import ceil
    per_page = rows * cols
    total = len(probs_list)
    pages = max(1, ceil(total / per_page))
    for pg in range(pages):
        s, e = pg * per_page, min((pg + 1) * per_page, total)
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2), squeeze=False)
        k = 0
        for r in range(rows):
            for c in range(cols):
                ax = axes[r, c]
                if s + k < e:
                    p = probs_list[s + k]
                    lab = labels[s + k]
                    fpr, tpr, _ = roc_curve(y_true01, p)
                    auc_val = auc(fpr, tpr)
                    ax.plot(fpr, tpr, color='purple',label=f"(AUC={auc_val:.2f})")
                    ax.plot([0, 1], [0, 1], linestyle="--")
                    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
                    ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
                    ax.legend(loc="lower right", fontsize=8)
                    ax.set_title(lab, fontsize=10)
                    k += 1
                else:
                    ax.axis("off")
        fig.suptitle(f"{title_prefix}  (models {s+1}-{e})", fontsize=14)
        fig.tight_layout(rect=[0, 0, 1, 0.96])
        out_png = os.path.join(OUTDIR, f"{out_prefix}_p{pg+1}.png")
        fig.savefig(out_png, dpi=150, bbox_inches="tight")
        plt.close(fig)
        print(f"[ROC] Saved subplot page: {out_png}")

# ---------------------------------------------
# Riskomon (memo) exporter helpers
# ---------------------------------------------
def _sigmoid(z: float) -> float:
    return 1.0 / (1.0 + math.exp(-z))

def _score_span(coefs: np.ndarray) -> Tuple[int, int]:
    """Min/max integer score contribution from sparse integer weights (no intercept)."""
    pos = int(coefs[coefs > 0].sum()) if coefs.size else 0
    neg = int(coefs[coefs < 0].sum()) if coefs.size else 0
    return neg, pos

def _risk_scale(multiplier: float, intercept: float, coefs: np.ndarray) -> List[List[float]]:
    """
    Returns [[score, prob], ...] with probability = sigmoid(multiplier * (intercept + score)).
    (Score here is the sum of integer feature weights; intercept is separate.)
    """
    smin, smax = _score_span(coefs)
    out = []
    for s in range(smin, smax + 1):
        total = intercept + s
        p = _sigmoid(multiplier * total)
        out.append([float(s), float(p)])
    return out

def _feature_pairs(coefs: np.ndarray, feat_names: List[str]) -> List[List[Any]]:
    """
    [[coef, "FeatureName"], ...] — coef first, only nonzeros, sorted by |coef| desc.
    """
    pairs = []
    for w, name in zip(coefs.tolist(), feat_names):
        if int(w) != 0:
            pairs.append([float(int(w)), str(name)])
    pairs.sort(key=lambda x: (-abs(x[0]), x[1]))
    return pairs

def export_riskomon_payload_memo(
    multipliers: List[float],
    intercepts: List[int],
    coef_matrix: List[np.ndarray],
    feature_names: List[str],
    X_train: np.ndarray, y_train: np.ndarray,
    dataset_tag: str = "CANCER_STLMD",
    export_n: int | None = None
) -> str:
    """
    Build JSON using memo schema:
      - feature_data: [[coef, "name"], ...]
      - risk_scale:   [[score, prob], ...]
      - training_logistic_loss: float
      - training_accuracy: float
      - training_AUC: float
      - card_label: "01", "02", ...
    """
    n_models = len(multipliers)
    use_n = min(export_n if isinstance(export_n, int) else n_models, n_models)

    payload = []
    width = max(2, len(str(use_n)))  # zero-pad like "01"
    for i in range(use_n):
        mult = float(multipliers[i])
        b0   = float(intercepts[i])
        betas = np.asarray(coef_matrix[i], dtype=int)

        # Use the native intercept for probability correctness.
        clf = RiskScoreClassifier(mult, b0, betas)
        clf.reset_featureNames(feature_names)

        train_loss = float(clf.compute_logisticLoss(X_train, y_train))
        train_acc, train_auc = clf.get_acc_and_auc(X_train, y_train)

        payload.append({
            "feature_data": _feature_pairs(betas, list(feature_names)),
            "risk_scale": _risk_scale(mult, b0, betas),
            "training_logistic_loss": train_loss,
            "training_accuracy": float(train_acc),
            "training_AUC": float(train_auc),
            "card_label": f"{i+1:0{width}d}",
        })

    out_fname = os.path.join(OUTDIR, f"{dataset_tag}.json")
    with open(out_fname, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    print(f"[Riskomon] Wrote {out_fname} with {use_n} models (memo schema)")
    return out_fname

# ---------------------------------------------
# Wire up your prepared splits & feature names
# (These are defined in your dataset block.)
# ---------------------------------------------
SPLITS = {
    "STLMD": (X_train_STLMD, y_train_STLMD, X_val_STLMD, y_val_STLMD, X_test_STLMD, y_test_STLMD, FEATURE_NAMES_STLMD),
    "STLM":  (X_train_STLM,  y_train_STLM,  X_val_STLM,  y_val_STLM,  X_test_STLM,  y_test_STLM,  FEATURE_NAMES_STLM),
    "STPGLM":(X_train_STPGLM,y_train_STPGLM,X_val_STPGLM,y_val_STPGLM,X_test_STPGLM,y_test_STPGLM,FEATURE_NAMES_STPGLM),
}

# ============================
# Train / evaluate / export per design
# ============================
for name in DESIGNS_TO_RUN:
    print(f"\n=== Processing: {name} ===")
    (X_train, y_train, X_val, y_val, X_test, y_test, feature_names) = SPLITS[name]

    # build optimizer with Rashomon controls
    opt = make_optimizer(
        X=X_train, y=y_train,
        k=SPARSITY_K, parent_size=PARENT_SIZE,
        gap_tolerance=GAP_TOLERANCE,
        select_top_m=SELECT_TOP_M,
        max_attempts=MAX_ATTEMPTS,
        want_intercept=True
    )
    print(f"[opt] gap_tolerance={GAP_TOLERANCE}, select_top_m={SELECT_TOP_M}, maxAttempts={MAX_ATTEMPTS}, k={SPARSITY_K}, parent_size={PARENT_SIZE}")

    t0 = time.time()
    opt.optimize()
    print("Optimization takes {:.2f} seconds.".format(time.time() - t0))

    multipliers, beta0_int, betas_int = extract_models(opt.get_models())
    print("We generate {} risk score models from the sparse diverse pool".format(len(multipliers)))

    # metrics + probs (TRAIN / TEST)
    train_aucs, train_accs, n_terms, train_probs = compute_model_metrics(
        multipliers, beta0_int, betas_int, X_train, y_train
    )
    test_aucs,  test_accs,  _,        test_probs  = compute_model_metrics(
        multipliers, beta0_int, betas_int, X_test,  y_test
    )

    # print features + AUCs per model
    print_model_summaries(multipliers, beta0_int, betas_int, feature_names, train_aucs, test_aucs)

    # save metrics CSV
    card_labels = [f"{i+1:02d}" for i in range(len(multipliers))]
    df_models = pd.DataFrame({
        "card": card_labels,
        "n_terms": n_terms,
        "train_auc": train_aucs,
        "test_auc":  test_aucs,
        "train_acc": train_accs,
        "test_acc":  test_accs,
        "intercept": beta0_int,
        "multiplier": multipliers,
    })
    csv_out = os.path.join(OUTDIR, f"model_metrics_{name}.csv")
    df_models.sort_values("test_auc", ascending=False).to_csv(csv_out, index=False)
    print(f"[AUC] Wrote {csv_out}")

    # ROC subplots for ALL models (TRAIN & TEST), 50 per page
    y_train01 = ((y_train + 1.0) / 2.0).astype(int)
    y_test01  = ((y_test  + 1.0) / 2.0).astype(int)

    plot_roc_subplots_all(
        y_true01=y_train01,
        probs_list=train_probs,
        labels=[f"Card {c}" for c in card_labels],
        rows=SUBPLOT_ROWS, cols=SUBPLOT_COLS,
        title_prefix=f"{name} — TRAIN ROC",
        out_prefix=f"roc_grid_{name}_train"
    )
    plot_roc_subplots_all(
        y_true01=y_test01,
        probs_list=test_probs,
        labels=[f"Card {c}" for c in card_labels],
        rows=SUBPLOT_ROWS, cols=SUBPLOT_COLS,
        title_prefix=f"{name} — TEST ROC",
        out_prefix=f"roc_grid_{name}_test"
    )

    # ---- Riskomon JSON export (memo-compatible) ----
    _ = export_riskomon_payload_memo(
        multipliers=multipliers,
        intercepts=beta0_int,
        coef_matrix=betas_int,
        feature_names=list(feature_names),
        X_train=X_train, y_train=y_train,
        dataset_tag=f"CANCER_{name}",
        export_n=None  # or an int like 50 if you want to cap
    )



=== Processing: STLMD ===
[opt] gap_tolerance=0.15, select_top_m=100, maxAttempts=200, k=5, parent_size=50
Optimization takes 135.32 seconds.
We generate 71 risk score models from the sparse diverse pool

=== Per-model summaries ===
[Card 01] b0=-7.000, mult=2.430, #terms=5
  Train AUC: 0.705 | Test AUC: 0.697
  Features : age >= 73(+3), sct_long_dia >= 8(+2), sct_long_dia >= 12(+2), solid(-1), Spiculation(+2)
[Card 02] b0=-12.000, mult=3.272, #terms=5
  Train AUC: 0.707 | Test AUC: 0.714
  Features : age >= 58(+2), age >= 73(+3), sct_long_dia >= 8(+3), sct_long_dia >= 12(+3), Spiculation(+2)
[Card 03] b0=-11.000, mult=3.519, #terms=5
  Train AUC: 0.708 | Test AUC: 0.717
  Features : age >= 58(+2), sct_long_dia >= 8(+3), sct_long_dia >= 12(+3), solid(-2), Spiculation(+2)
[Card 04] b0=-11.000, mult=3.977, #terms=5
  Train AUC: 0.704 | Test AUC: 0.697
  Features : age >= 73(+4), sct_long_dia >= 8(+4), sct_long_dia >= 15(+3), solid(-2), Spiculation(+3)
[Card 05] b0=-10.000, mult=2.980, #

[ROC] Saved subplot page: ./FasterRisk_STLMD_Outputs/roc_grid_STLMD_train_p1.png
[ROC] Saved subplot page: ./FasterRisk_STLMD_Outputs/roc_grid_STLMD_train_p2.png
[ROC] Saved subplot page: ./FasterRisk_STLMD_Outputs/roc_grid_STLMD_test_p1.png
[ROC] Saved subplot page: ./FasterRisk_STLMD_Outputs/roc_grid_STLMD_test_p2.png
[Riskomon] Wrote ./FasterRisk_STLMD_Outputs/CANCER_STLMD.json with 71 models (memo schema)
